In [11]:
#Import standard library, NLTK and sklearn
import pandas as pd
import numpy as np
import os
import re
import operator
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer


In [12]:
#Loading json file news
news = pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/newsgroups.json')

In [13]:

news

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [14]:
news.head

<bound method NDFrame.head of                                                  content  target  \
0      From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1      From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2      From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3      From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4      From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   
...                                                  ...     ...   
11309  From: jim.zisfein@factory.com (Jim Zisfein) \n...      13   
11310  From: ebodin@pearl.tufts.edu\nSubject: Screen ...       4   
11311  From: westes@netcom.com (Will Estes)\nSubject:...       3   
11312  From: steve@hcrlgw (Steven Collins)\nSubject: ...       1   
11313  From: gunning@cco.caltech.edu (Kevin J. Gunnin...       8   

                   target_names  
0                     rec.autos  
1         comp.sys.mac.hardware  
2         comp.sys.mac.hardware  
3                

In [16]:
#Data processing: Merangkum news bagian content dan subject
for i,txt in enumerate(news['content']):
    #print(i)
    subject = re.findall('Subject:(.*\n)',txt)
    if (len(subject) !=0):
        news.loc[i,'Subject'] =str(i)+' '+subject[0]
    else:
        news.loc[i,'Subject'] ='NA'


In [17]:

df_news =news[['Subject','content']]

In [18]:

df_news.head(5)

,Subject,content
0,0 WHAT car is this!?\n,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,1 SI Clock Poll - Final Call\n,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,2 PB questions...\n,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,3 Re: Weitek P9000 ?\n,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,4 Re: Shuttle Launch Question\n,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [19]:
#Data processisng: Mengubah menjadi huruf kecil
df_news['content']=[entry.lower() for entry in df_news['content']]

<ipython-input-19-94e80c480094>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['content']=[entry.lower() for entry in df_news['content']]


In [20]:
#menghapus kata-kata yang tidak bermakna dan sering berulang
SW=['subject:','organization:','thanks','thank','re:']
for sw in SW:
    df_news.content=df_news.content.replace(to_replace=sw,value='',regex=True)

<ipython-input-20-ff55cac65ee3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content=df_news.content.replace(to_replace=sw,value='',regex=True)


In [21]:
## Data processing: data Cleaning for content data of news
df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True) #remove from to email
df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_news.content =df_news.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_news.content =df_news.content.replace(to_replace='-',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.content =df_news.content.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_news.content =df_news.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

<ipython-input-21-aa6b4b110b08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True) #remove from to email
<ipython-input-21-aa6b4b110b08>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
<ipython-input-21-aa6b4b110b08>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [22]:
## data cleaning for subject data
df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',value=' ',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.Subject =df_news.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
df_news.Subject =df_news.Subject.apply(lambda x:x.strip())


<ipython-input-22-25cd4a6ebf4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
<ipython-input-22-25cd4a6ebf4c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',value=' ',regex=True)
<ipython-input-22-25cd4a6ebf4c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [26]:
## Data processing Checking  and drop empty data
for i,sen in enumerate(df_news.content):
    if len(sen.strip()) ==0:
        print(str(i))
        #file_data.text[i] = np.nan
        df_news=df_news.drop(str(i),axis=0).reset_index().drop('index',axis=1)

In [27]:

#download punkt
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
# tokenization : In this each entry in the file_data will be broken into set of words
df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]

<ipython-input-28-5397cfebd83c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]


In [29]:
#Advanced data processing
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):

        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:

df_news.shape


(11314, 3)

In [31]:

df_news.head(2)

,Subject,content,Word tokenize
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,..."


In [32]:
## This Function took around 13 hours for word Lemmatized and remove the Stop words & single character of word of each 11314 rows.
df_clean = wordLemmatizer(df_news['Word tokenize'][0:1000])
df_clean

,Keyword_final
0,"['car', 'nntp', 'post', 'host', 'university', ..."
1,"['si', 'clock', 'poll', 'final', 'call', 'summ..."
2,"['pb', 'question', 'purdue', 'university', 'en..."
3,"['weitek', 'harris', 'computer', 'system', 'di..."
4,"['shuttle', 'launch', 'question', 'smithsonian..."
...,...
995,"['terminal', 'sale', 'nyx', 'public', 'access'..."
996,"['remember', 'name', 'come', 'election', 'time..."
997,"['interest', 'adb', 'behaviour', 'nntp', 'post..."
998,"['request', 'information', 'essential', 'tremo..."


In [33]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [34]:
# ambil yang sudah di process (karena proses sendiri lama)
df =pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/WordLemmatize20NewsGroup.json')
df_news['Clean_Keyword'] =df['Clean_Keyword']

In [35]:

df_news.head(5)

,Subject,content,Word tokenize,Clean_Keyword
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac...","thing,car,nntp,post,host,university,maryland,c..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,...","guy,kuo,si,clock,poll,final,call,summary,final..."
2,2 PB questions,pb questions... purdue university engineering ...,"[pb, questions, ..., purdue, university, engin...","thomas,willis,pb,question,purdue,university,en..."
3,3 Weitek P9000,weitek p9000 harris computer systems division ...,"[weitek, p9000, harris, computer, systems, div...","joe,green,weitek,harris,computer,system,divisi..."
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...,"[shuttle, launch, question, smithsonian, astro...","jonathan,mcdowell,shuttle,launch,question,smit..."


In [36]:
#hanya mengambil content
df_news_save= df_news
df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_news_save

,Subject,content
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...
2,2 PB questions,pb questions... purdue university engineering ...
3,3 Weitek P9000,weitek p9000 harris computer systems division ...
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...
...,...,...
11309,11309 Migraines and scans,migraines and scans distribution world inventi...
11310,11310 Screen Death Mac Plus 512,screen death mac plus 512 tufts university med...
11311,11311 Mounting CPU Cooler in vertical case,mounting cpu cooler in vertical case mail grou...
11312,11312 Sphere from 4 points,sphere from 4 points central research lab. hit...


In [37]:

df_news_save.to_csv("df_news_index.csv", index=False, header=True)

In [38]:
#kita intip berita pertama: sudah bersih dan terpotong-potong
df_news.Clean_Keyword[0]

'thing,car,nntp,post,host,university,maryland,college,park,line,wonder,anyone,could,enlighten,car,saw,day,sport,car,look,late,early,call,bricklin,door,really,small,addition,front,bumper,separate,rest,body,know,anyone,tellme,model,name,engine,spec,year,production,car,make,history,whatever,info,funky,look,car,please,mail,il,bring,neighborhood,lerxst'

In [39]:

#codingan inti TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df_news.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

# Fit the TfIdf model
tfidf.fit(df_news.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_news.Clean_Keyword)

In [40]:

vocabulary[0:4]

['rant', 'yearsley', 'harpy', 'kd']

In [41]:

### Save model
with open('/content/tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [42]:
### load model
t = pickle.load(open('/content/tfid.pkl','rb'))

In [43]:
### Save Vacabulary
with open("/content/vocabulary_news20group.txt", "w") as file:
    file.write(str(vocabulary))

In [44]:

### load Vacabulary
with open("/content/vocabulary_news20group.txt", "r") as file:
    data2 = eval(file.readline())

In [45]:
print(data2[0:10])

['rant', 'yearsley', 'harpy', 'kd', 'caneven', 'mmzvm', 'righty', 'ifip', 'rjungcla', 'pile']


In [46]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))

    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [47]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim


In [48]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    #print("\nQuery:", query)
    #print("")
    #print(tokens)

    d_cosines = []

    query_vector = gen_vector_T(q_df['q_clean'])

    for d in tfidf_tran.A:

        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_news['Subject'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a


In [49]:
#CONTOH Aplikasi, mencari document dengan keyword Politic Election
%time cosine_similarity_T(10,'politic election')


CPU times: user 2.54 s, sys: 4.55 s, total: 7.09 s
Wall time: 6.53 s


,index,Subject,Score
0,10029,10029 Celebrate Liberty 1993,0.161847
1,3384,3384 Temper tantrums from the 1960 s,0.154731
2,10604,10604 Temper tantrums from the 1960 s,0.144369
3,4846,4846 Questions to Ponder,0.141771
4,6632,6632 BMW MOA members read this,0.112939
5,10655,10655 Help Disappearing Groups,0.105913
6,1945,1945 BMW MOA members read this,0.085879
7,9433,9433 PUBLIC HEARINGS on Ballot Access Vote Fra...,0.082890
8,3824,3824 Remember those names come election time,0.075318
9,3728,3728 Hilter and homosexuals,0.074864


In [50]:

#ternyata yang paling dekat dengan keyword politic election adalah dokumen nomor 2231. Sekarang kita lihat isi dokumen nya
df_news.content[2231]

'computer engr vs. computer science distribution usa ncr microelectronics products division an at t company in article 1993apr10.210242.340 macadam.com mike macadam.com writes i am a freshman in college and can t decide whether to major in computer engineering or computer science. any advice or suggestions will be appreciated. mike a professor of mine once said the difference between a computer engineer and a computer scientist is about 5000 meaning the engineer makes 5000 more than the cs. seriously though the main difference is that most cs people write programs that people will use i.e. database graphics word processors etc. while an engineer writes for machines or control systems i.e. the computer in your car a flight control system computer controled devices etc. in other words cs writes software while cse writes firmware. these are generalizations but for the most part that is what the difference is. p.s. the 5000 is not just a joke scott'

In [51]:
#Kita search dengan keyword food industry
%time cosine_similarity_T(5,'food industry')

CPU times: user 2.55 s, sys: 3.69 s, total: 6.23 s
Wall time: 5.15 s


,index,Subject,Score
0,2214,2214 Is MSG sensitivity superstition,0.522039
1,4378,4378 Is MSG sensitivity superstition,0.395787
2,8331,8331 Is MSG sensitivity superstition,0.374125
3,6003,6003 Barbecued foods and health risk,0.344936
4,612,612 Is MSG sensitivity superstition,0.303564


In [52]:

#intip hasil dokumen paling atas pencarian
df_news.content[3948]


'switch mode power supply suite software reply to suite tarbell uunet.uu.net nntp posting host gilgamesh.suite.com is there a typical component or set of components that are at fault when a switch mode power supply goes south the supply is for a disk drive. any general hints would be appreciated mark tarbell suite.com at fault when a switch mode power supply goes south the supply is for a disk drive. any general hints would be appreciated mark tarbell s \x03\x03\x1b bvh\x1b j\x06'